# NIRCam Preimaging: Pipeline Stages 1, 2

**Use case:** Running JWST Pipeline on NIRCam Preimaging Simulations.<br>
**Data:** JWST simulated NIRCam data from MIRAGE; LMC.<br>
**Tools:**  jwst.<br>
**Cross-intrument:** NIRCam. <br>
**Documentation:** This notebook is part of a STScI's larger [post-pipeline Data Analysis Tools Ecosystem](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis).<br>

## Introduction

This Notebook must be executed from an environment that has the JWST pipeline installed. In this tutorial we show step by step instructions on how to run the JWST pipeline on the simulated images created using the previous notebook.

This notebook executes the first two steps of the NIRCam pipeline on the images that were created with the simulator MIRAGE in `preimaging_01_mirage`. The example shows how to execute the pipeline on the F150W images only. 

## Setting things up

In [1]:
import os

#os.environ['CRDS_PATH'] = os.environ['HOME']+'/crds_cache'
os.environ['CRDS_PATH'] = '/ifs/jwst/wit/miri/ofox/preimaging/cache'

os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds-pub.stsci.edu'

In [2]:
from sys import exit
import json
from glob import glob

import numpy as np

from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import calwebb_image2

from astropy.table import Table
from astropy.io import fits



In [3]:
path='./.' # this is the working directory
os.chdir(path)
print(os.getcwd())

/home/runner/work/jdat_notebooks/jdat_notebooks/notebooks/preimaging


In [4]:
filtname = 'f150w'

In [5]:
# gather the F150W yaml files back in the working directory
cwd = os.getcwd()
filter_pattern = os.path.join(cwd, 'yaml', filtname.lower(), 'jw01069001001*0004*yaml') 
files = glob(filter_pattern)[:]
inputyaml = files
namelist = []
outlist = []

We define the calwebb_detector1 configuration file to use in the pipeline call.

In [6]:
calwebb_detector1_file = os.path.join('preimaging', 'calwebb_detector1.cfg')

## Executing calwebb_detector1

We select the *_uncal.fits files and execute calwebb_detector1 to generate the *_rate.fits files.

In [7]:
for input_yaml_path in inputyaml:
    print(input_yaml_path) 
    
    base = os.path.basename(input_yaml_path)
    name = os.path.splitext(base)[0]

    exi = os.path.exists(name + '_rate.fits')
    if exi:
        continue
        
    namelist.append(name + '_uncal.fits')
    outlist.append(name + '.fits')
    
print(namelist)
print(outlist)

files  = namelist 
outfile = outlist 


for k, filename in enumerate(files):
    Detector1Pipeline.call(filename, output_file=outfile[k]);


[]
[]


## Executing calwebb_image2

In [8]:
# gather the F150W yaml files back in the working directory
cwd = os.getcwd()
filter_pattern = os.path.join(cwd, 'yaml', filtname.lower(), 'jw01069001001*0004*') 
files = glob(filter_pattern)[:] # Use [0:2] to test the first two files.
inputyaml = files
namelist = []
outlist = []

We select the *_rate.fits files and execute calwebb_image2 to generate the *_cal.fits files.

In [9]:
for input_yaml_path in inputyaml:
    print(input_yaml_path)
    base = os.path.basename(input_yaml_path)
    name = os.path.splitext(base)[0]
   
    exi = os.path.exists(name + '_cal.fits')
    if exi:
        continue

    namelist.append(name + '_rate.fits')
    outlist.append(name + '.fits')
        
print(namelist)
    
files = namelist 
outfile = outlist 

[]


Generate the _cal.fits files.

In [10]:
for k, filename in enumerate(files):
    print('-'*60)
    print(filename)

    m  =  calwebb_image2.Image2Pipeline(output_file=outfile[k])
    m.run(filename)
   